In [1]:
Steps:

1. Read the PDF file 

2. Convert the PDF file into small chunk

3. Create a simplified prompt using llamaIndex - This we will use multi prompt

4. Splitting the document and create english embeddings 

5. Loading the data into Chroma DB

6. Summarization of the whole document - Matching with similarity index of 5. 


SyntaxError: invalid syntax (563097739.py, line 1)

In [2]:
pip install llama_index llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-vector-stores-chroma llama-index-llms-mistralai qdrant-client mistralai

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pypdf2

Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import PyPDF2

# Define the PDF and text file paths
pdf_file = "./bookpdf/AtomicHabits.pdf"
text_file = "./atomichabit/extracted_atomichabit.txt"
text = ""

# Open the PDF file
with open(pdf_file, 'rb') as f:
  pdf_reader = PyPDF2.PdfReader(f)
    # Extract text and store it in a variable
  for page in pdf_reader.pages:
    text += page.extract_text()

# Open the text file in write mode and write the extracted text
with open(text_file, "w") as f:
  f.write(text)

print(f"Text extracted from {pdf_file} and saved to {text_file}")

Text extracted from ./bookpdf/AtomicHabits.pdf and saved to ./atomichabit/extracted_atomichabit.txt


In [5]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

collection_name = "bookparser"
chroma_client = chromadb.PersistentClient()

# Hypothetical method to list all collections. The actual method name may vary.
collections = chroma_client.list_collections()

# Check if the collection exists by looking through the list of collections
if any(collection.name == collection_name for collection in collections):
    # Proceed to delete the collection
    try:
        chroma_client.delete_collection(collection_name)
        print(f"Collection '{collection_name}' has been deleted.")
    except Exception as e:
        print(f"Failed to delete collection '{collection_name}': {e}")

chroma_collection = chroma_client.create_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Collection 'bookparser' has been deleted.


In [6]:
# Global settings
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# Local settings
from llama_index.core.node_parser import SentenceSplitter

Settings.chunk_size = 512
documents = SimpleDirectoryReader("atomichabit").load_data()

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,  transformations=[SentenceSplitter(chunk_size=512)]
)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author talk about?")
print(response)

The author discussed his journey from personal experiments with habits to sharing them publicly through articles on his website, which eventually led to a significant increase in email subscribers. This success in writing about habits led to business opportunities, speaking engagements at companies, and the eventual publication of the book the reader is currently engaged with.
